# FixSigMeta

The goal of this notebook is to explore how `FixSigMeta` avoid problems when `Foo` wants to get signature from its `__init__` using `inspect.signature`. 

In order to understand the problems `FixSigMeta` is fixing, please read official [docs](https://fastcore.fast.ai/meta.html#fixsigmeta) first

At the beginning, I know very little of the source code of `inspect.signature` or in fact `inspect._signature_from_callable` and have no idea how `FixSigMeta` enable `Foo` to overcome the potential problems.

`Fastdebug` library and its `Fastdb.dbprint` enables me to debug any source code and evaluate the expressions you write sitting above the source code and `Fastdb.print` can display source code with comments I add when debugging with `dbprint`.

At the end of the notebook, I hope to have a nice and detailed document on the exploration and have a in-depth understanding of how both `_signature_from_callable` and `FixSigMeta` work.

Here is what I learnt from this notebook about how `FixSigMeta` solve the potential problems 


> As a metaclass, `FixSigMeta` defines its `__new__` to creates a class instance `Foo` with attribute `__signature__` and store the signature of `__init__`. This way `inspect._signature_from_callable` can directly help `Foo` to get signature from `__signature__` instead of going into `__new__`, `__call__` looking for signatures where the potential problems reside.

## Having a better view in Jupyter notebook

In [1]:
# from IPython.core.display import display, HTML # a depreciated import
from IPython.display import display, HTML 

In [2]:
display(HTML("<style>.container { width:100% !important; }</style>"))

## Examples to explore

In [3]:
import inspect

### `Foo` borrows signature from `Foo.__init__`

In [4]:
class Foo:
    def __init__(self, a, b, c): pass

    @classmethod
    def clsmed(): pass
    
inspect.signature(Foo)

<Signature (a, b, c)>

### `Base.__new__` stops `Foo` borrows signature from `Foo.__init__`

In [5]:
class Base: # pass
    def __new__(self, **args): pass  # defines a __new__ 

class Foo(Base):
    def __init__(self, d, e, f): pass
    
inspect.signature(Foo) # not a problem for python 3.9+, but is a problem for python 3.7

<Signature (d, e, f)>

In [7]:
sigOld="""
def _signature_from_callableOld(obj, *,
                             follow_wrapper_chains=True,
                             skip_bound_arg=True,
                             sigcls):

    "Private helper function to get signature for arbitrary callable objects."

    if not callable(obj):
        raise TypeError('{!r} is not a callable object'.format(obj))

    if isinstance(obj, types.MethodType):
        # In this case we skip the first parameter of the underlying
        # function (usually `self` or `cls`).
        sig = _signature_from_callable(
            obj.__func__,
            follow_wrapper_chains=follow_wrapper_chains,
            skip_bound_arg=skip_bound_arg,
            sigcls=sigcls)

        if skip_bound_arg:
            return _signature_bound_method(sig)
        else:
            return sig

    # Was this function wrapped by a decorator?
    if follow_wrapper_chains:
        obj = unwrap(obj, stop=(lambda f: hasattr(f, "__signature__")))
        if isinstance(obj, types.MethodType):
            # If the unwrapped object is a *method*, we might want to
            # skip its first parameter (self).
            # See test_signature_wrapped_bound_method for details.
            return _signature_from_callable(
                obj,
                follow_wrapper_chains=follow_wrapper_chains,
                skip_bound_arg=skip_bound_arg,
                sigcls=sigcls)

    try:
        sig = obj.__signature__
    except AttributeError:
        pass
    else:
        if sig is not None:
            if not isinstance(sig, Signature):
                raise TypeError(
                    'unexpected object {!r} in __signature__ '
                    'attribute'.format(sig))
            return sig

    try:
        partialmethod = obj._partialmethod
    except AttributeError:
        pass
    else:
        if isinstance(partialmethod, functools.partialmethod):
            # Unbound partialmethod (see functools.partialmethod)
            # This means, that we need to calculate the signature
            # as if it's a regular partial object, but taking into
            # account that the first positional argument
            # (usually `self`, or `cls`) will not be passed
            # automatically (as for boundmethods)

            wrapped_sig = _signature_from_callable(
                partialmethod.func,
                follow_wrapper_chains=follow_wrapper_chains,
                skip_bound_arg=skip_bound_arg,
                sigcls=sigcls)

            sig = _signature_get_partial(wrapped_sig, partialmethod, (None,))
            first_wrapped_param = tuple(wrapped_sig.parameters.values())[0]
            if first_wrapped_param.kind is Parameter.VAR_POSITIONAL:
                # First argument of the wrapped callable is `*args`, as in
                # `partialmethod(lambda *args)`.
                return sig
            else:
                sig_params = tuple(sig.parameters.values())
                assert (not sig_params or
                        first_wrapped_param is not sig_params[0])
                new_params = (first_wrapped_param,) + sig_params
                return sig.replace(parameters=new_params)

    if isfunction(obj) or _signature_is_functionlike(obj):
        # If it's a pure Python function, or an object that is duck type
        # of a Python function (Cython functions, for instance), then:
        return _signature_from_function(sigcls, obj)

    if _signature_is_builtin(obj):
        return _signature_from_builtin(sigcls, obj,
                                       skip_bound_arg=skip_bound_arg)

    if isinstance(obj, functools.partial):
        wrapped_sig = _signature_from_callable(
            obj.func,
            follow_wrapper_chains=follow_wrapper_chains,
            skip_bound_arg=skip_bound_arg,
            sigcls=sigcls)
        return _signature_get_partial(wrapped_sig, obj)

    sig = None
    if isinstance(obj, type):
        # obj is a class or a metaclass

        # First, let's see if it has an overloaded __call__ defined
        # in its metaclass
        call = _signature_get_user_defined_method(type(obj), '__call__')
        if call is not None:
            sig = _signature_from_callable(
                call,
                follow_wrapper_chains=follow_wrapper_chains,
                skip_bound_arg=skip_bound_arg,
                sigcls=sigcls)
        else:
            # Now we check if the 'obj' class has a '__new__' method
            new = _signature_get_user_defined_method(obj, '__new__')
            if new is not None:
                sig = _signature_from_callable(
                    new,
                    follow_wrapper_chains=follow_wrapper_chains,
                    skip_bound_arg=skip_bound_arg,
                    sigcls=sigcls)
            else:
                # Finally, we should have at least __init__ implemented
                init = _signature_get_user_defined_method(obj, '__init__')
                if init is not None:
                    sig = _signature_from_callable(
                        init,
                        follow_wrapper_chains=follow_wrapper_chains,
                        skip_bound_arg=skip_bound_arg,
                        sigcls=sigcls)

        if sig is None:
            # At this point we know, that `obj` is a class, with no user-
            # defined '__init__', '__new__', or class-level '__call__'

            for base in obj.__mro__[:-1]:
                # Since '__text_signature__' is implemented as a
                # descriptor that extracts text signature from the
                # class docstring, if 'obj' is derived from a builtin
                # class, its own '__text_signature__' may be 'None'.
                # Therefore, we go through the MRO (except the last
                # class in there, which is 'object') to find the first
                # class with non-empty text signature.
                try:
                    text_sig = base.__text_signature__
                except AttributeError:
                    pass
                else:
                    if text_sig:
                        # If 'obj' class has a __text_signature__ attribute:
                        # return a signature based on it
                        return _signature_fromstr(sigcls, obj, text_sig)

            # No '__text_signature__' was found for the 'obj' class.
            # Last option is to check if its '__init__' is
            # object.__init__ or type.__init__.
            if type not in obj.__mro__:
                # We have a class (not metaclass), but no user-defined
                # __init__ or __new__ for it
                if (obj.__init__ is object.__init__ and
                    obj.__new__ is object.__new__):
                    # Return a signature of 'object' builtin.
                    return sigcls.from_callable(object)
                else:
                    raise ValueError(
                        'no signature found for builtin type {!r}'.format(obj))

    elif not isinstance(obj, _NonUserDefinedCallables):
        # An object with __call__
        # We also check that the 'obj' is not an instance of
        # _WrapperDescriptor or _MethodWrapper to avoid
        # infinite recursion (and even potential segfault)
        call = _signature_get_user_defined_method(type(obj), '__call__')
        if call is not None:
            try:
                sig = _signature_from_callable(
                    call,
                    follow_wrapper_chains=follow_wrapper_chains,
                    skip_bound_arg=skip_bound_arg,
                    sigcls=sigcls)
            except ValueError as ex:
                msg = 'no signature found for {!r}'.format(obj)
                raise ValueError(msg) from ex

    if sig is not None:
        # For classes and objects we skip the first parameter of their
        # __call__, __new__, or __init__ methods
        if skip_bound_arg:
            return _signature_bound_method(sig)
        else:
            return sig

    if isinstance(obj, types.BuiltinFunctionType):
        # Raise a nicer error message for builtins
        msg = 'no signature found for builtin function {!r}'.format(obj)
        raise ValueError(msg)

    raise ValueError('callable {!r} is not supported by signature'.format(obj))
"""

In [8]:
inspect.__dict__

{'__name__': 'inspect',
 '__doc__': "Get useful information from live Python objects.\n\nThis module encapsulates the interface provided by the internal special\nattributes (co_*, im_*, tb_*, etc.) in a friendlier fashion.\nIt also provides some help for examining source code and class layout.\n\nHere are some of the useful functions provided by this module:\n\n    ismodule(), isclass(), ismethod(), isfunction(), isgeneratorfunction(),\n        isgenerator(), istraceback(), isframe(), iscode(), isbuiltin(),\n        isroutine() - check object types\n    getmembers() - get members of an object that satisfy a given condition\n\n    getfile(), getsourcefile(), getsource() - find an object's source code\n    getdoc(), getcomments() - get documentation on an object\n    getmodule() - determine the module that an object came from\n    getclasstree() - arrange classes so as to represent their hierarchy\n\n    getargvalues(), getcallargs() - get info about function arguments\n    getfullargspe

In [13]:
import ast

In [14]:
block = ast.parse(sigOld, mode='exec')
exec(compile(block, '<string>', mode='exec'), globals().update(inspect.__dict__))

In [15]:
list(locals().keys())

['__name__',
 '__doc__',
 '__package__',
 '__loader__',
 '__spec__',
 '__builtin__',
 '__builtins__',
 '_ih',
 '_oh',
 '_dh',
 'In',
 'Out',
 'get_ipython',
 'exit',
 'quit',
 '_',
 '__',
 '___',
 '_i',
 '_ii',
 '_iii',
 '_i1',
 'display',
 'HTML',
 '_i2',
 '_i3',
 'inspect',
 '_i4',
 'Foo',
 '_4',
 '_i5',
 'Base',
 '_5',
 '_i6',
 '_i7',
 'sigOld',
 '_i8',
 '_8',
 '_i9',
 '_i10',
 '_10',
 '_i11',
 '_11',
 '_i12',
 '_12',
 '_i13',
 'ast',
 '_i14',
 'block',
 '__file__',
 '__cached__',
 '__author__',
 'abc',
 'dis',
 'collections',
 'enum',
 'importlib',
 'itertools',
 'linecache',
 'os',
 're',
 'sys',
 'tokenize',
 'token',
 'types',
 'warnings',
 'functools',
 'builtins',
 'attrgetter',
 'namedtuple',
 'OrderedDict',
 'mod_dict',
 'k',
 'v',
 'CO_OPTIMIZED',
 'CO_NEWLOCALS',
 'CO_VARARGS',
 'CO_VARKEYWORDS',
 'CO_NESTED',
 'CO_GENERATOR',
 'CO_NOFREE',
 'CO_COROUTINE',
 'CO_ITERABLE_COROUTINE',
 'CO_ASYNC_GENERATOR',
 'TPFLAGS_IS_ABSTRACT',
 'ismodule',
 'isclass',
 'ismethod',
 'isme

In [17]:
_signature_from_callableOld?

In [19]:
_signature_from_callable?

In [18]:
inspect._signature_from_callable = _signature_from_callableOld
inspect.signature(Foo) # not a problem for python 3.9+, but is a problem for python 3.7

<Signature (**args)>

## Imports

In [7]:
from fastdebug.core import *

In [8]:
import inspect
from inspect import *
from inspect import _signature_from_callable
from inspect import _signature_is_functionlike, _signature_is_builtin, _signature_get_user_defined_method, _signature_from_function, _signature_bound_method

In [9]:
import fastcore.meta as fm

In [10]:
_signature_from_callableNew = _signature_from_callable

In [11]:
def _signature_from_callableOld(obj, *,
                             follow_wrapper_chains=True,
                             skip_bound_arg=True,
                             sigcls):

    """Private helper function to get signature for arbitrary
    callable objects.
    """

    if not callable(obj):
        raise TypeError('{!r} is not a callable object'.format(obj))

    if isinstance(obj, types.MethodType):
        # In this case we skip the first parameter of the underlying
        # function (usually `self` or `cls`).
        sig = _signature_from_callable(
            obj.__func__,
            follow_wrapper_chains=follow_wrapper_chains,
            skip_bound_arg=skip_bound_arg,
            sigcls=sigcls)

        if skip_bound_arg:
            return _signature_bound_method(sig)
        else:
            return sig

    # Was this function wrapped by a decorator?
    if follow_wrapper_chains:
        obj = unwrap(obj, stop=(lambda f: hasattr(f, "__signature__")))
        if isinstance(obj, types.MethodType):
            # If the unwrapped object is a *method*, we might want to
            # skip its first parameter (self).
            # See test_signature_wrapped_bound_method for details.
            return _signature_from_callable(
                obj,
                follow_wrapper_chains=follow_wrapper_chains,
                skip_bound_arg=skip_bound_arg,
                sigcls=sigcls)

    try:
        sig = obj.__signature__
    except AttributeError:
        pass
    else:
        if sig is not None:
            if not isinstance(sig, Signature):
                raise TypeError(
                    'unexpected object {!r} in __signature__ '
                    'attribute'.format(sig))
            return sig

    try:
        partialmethod = obj._partialmethod
    except AttributeError:
        pass
    else:
        if isinstance(partialmethod, functools.partialmethod):
            # Unbound partialmethod (see functools.partialmethod)
            # This means, that we need to calculate the signature
            # as if it's a regular partial object, but taking into
            # account that the first positional argument
            # (usually `self`, or `cls`) will not be passed
            # automatically (as for boundmethods)

            wrapped_sig = _signature_from_callable(
                partialmethod.func,
                follow_wrapper_chains=follow_wrapper_chains,
                skip_bound_arg=skip_bound_arg,
                sigcls=sigcls)

            sig = _signature_get_partial(wrapped_sig, partialmethod, (None,))
            first_wrapped_param = tuple(wrapped_sig.parameters.values())[0]
            if first_wrapped_param.kind is Parameter.VAR_POSITIONAL:
                # First argument of the wrapped callable is `*args`, as in
                # `partialmethod(lambda *args)`.
                return sig
            else:
                sig_params = tuple(sig.parameters.values())
                assert (not sig_params or
                        first_wrapped_param is not sig_params[0])
                new_params = (first_wrapped_param,) + sig_params
                return sig.replace(parameters=new_params)

    if isfunction(obj) or _signature_is_functionlike(obj):
        # If it's a pure Python function, or an object that is duck type
        # of a Python function (Cython functions, for instance), then:
        return _signature_from_function(sigcls, obj)

    if _signature_is_builtin(obj):
        return _signature_from_builtin(sigcls, obj,
                                       skip_bound_arg=skip_bound_arg)

    if isinstance(obj, functools.partial):
        wrapped_sig = _signature_from_callable(
            obj.func,
            follow_wrapper_chains=follow_wrapper_chains,
            skip_bound_arg=skip_bound_arg,
            sigcls=sigcls)
        return _signature_get_partial(wrapped_sig, obj)

    sig = None
    if isinstance(obj, type):
        # obj is a class or a metaclass

        # First, let's see if it has an overloaded __call__ defined
        # in its metaclass
        call = _signature_get_user_defined_method(type(obj), '__call__')
        if call is not None:
            sig = _signature_from_callable(
                call,
                follow_wrapper_chains=follow_wrapper_chains,
                skip_bound_arg=skip_bound_arg,
                sigcls=sigcls)
        else:
            # Now we check if the 'obj' class has a '__new__' method
            new = _signature_get_user_defined_method(obj, '__new__')
            if new is not None:
                sig = _signature_from_callable(
                    new,
                    follow_wrapper_chains=follow_wrapper_chains,
                    skip_bound_arg=skip_bound_arg,
                    sigcls=sigcls)
            else:
                # Finally, we should have at least __init__ implemented
                init = _signature_get_user_defined_method(obj, '__init__')
                if init is not None:
                    sig = _signature_from_callable(
                        init,
                        follow_wrapper_chains=follow_wrapper_chains,
                        skip_bound_arg=skip_bound_arg,
                        sigcls=sigcls)

        if sig is None:
            # At this point we know, that `obj` is a class, with no user-
            # defined '__init__', '__new__', or class-level '__call__'

            for base in obj.__mro__[:-1]:
                # Since '__text_signature__' is implemented as a
                # descriptor that extracts text signature from the
                # class docstring, if 'obj' is derived from a builtin
                # class, its own '__text_signature__' may be 'None'.
                # Therefore, we go through the MRO (except the last
                # class in there, which is 'object') to find the first
                # class with non-empty text signature.
                try:
                    text_sig = base.__text_signature__
                except AttributeError:
                    pass
                else:
                    if text_sig:
                        # If 'obj' class has a __text_signature__ attribute:
                        # return a signature based on it
                        return _signature_fromstr(sigcls, obj, text_sig)

            # No '__text_signature__' was found for the 'obj' class.
            # Last option is to check if its '__init__' is
            # object.__init__ or type.__init__.
            if type not in obj.__mro__:
                # We have a class (not metaclass), but no user-defined
                # __init__ or __new__ for it
                if (obj.__init__ is object.__init__ and
                    obj.__new__ is object.__new__):
                    # Return a signature of 'object' builtin.
                    return sigcls.from_callable(object)
                else:
                    raise ValueError(
                        'no signature found for builtin type {!r}'.format(obj))

    elif not isinstance(obj, _NonUserDefinedCallables):
        # An object with __call__
        # We also check that the 'obj' is not an instance of
        # _WrapperDescriptor or _MethodWrapper to avoid
        # infinite recursion (and even potential segfault)
        call = _signature_get_user_defined_method(type(obj), '__call__')
        if call is not None:
            try:
                sig = _signature_from_callable(
                    call,
                    follow_wrapper_chains=follow_wrapper_chains,
                    skip_bound_arg=skip_bound_arg,
                    sigcls=sigcls)
            except ValueError as ex:
                msg = 'no signature found for {!r}'.format(obj)
                raise ValueError(msg) from ex

    if sig is not None:
        # For classes and objects we skip the first parameter of their
        # __call__, __new__, or __init__ methods
        if skip_bound_arg:
            return _signature_bound_method(sig)
        else:
            return sig

    if isinstance(obj, types.BuiltinFunctionType):
        # Raise a nicer error message for builtins
        msg = 'no signature found for builtin function {!r}'.format(obj)
        raise ValueError(msg)

    raise ValueError('callable {!r} is not supported by signature'.format(obj))

## Prepare environment variables for debugging

In [12]:
len(dir(fm))

103

In [13]:
len(inspect.__dict__)

167

In [14]:
g = {}
# g.update(inspect.__dict__)
# g.update(fm.__dict__)
g.update(inspect._signature_from_callable.__globals__)
g.update(fm.delegates.__globals__)

len(g)

254

## When or why to use `FixSigMeta`?

When we want a class e.g., `Foo` to have signature from its `__init__` method.

`FixSigMeta` can avoid potential problems for `Foo` to access signature from `__init__`.

In [15]:
class Foo:
    def __init__(self, a, b, c): pass

    @classmethod
    def clsmed(): pass
    
inspect.signature(Foo)

<Signature (a, b, c)>

## How Foo borrow sig from `__init__`

In [16]:
# g = locals()
# sig = Fastdb(_signature_from_callable, g)
sig = Fastdb(_signature_from_callable)

### How to debug `inspect._signature_from_callable` with Fastdb

In [17]:
#| column: screen

sig.dbprint(9, "so that it can use in itself")
sig.print(part=1)

    """                                                                                                                                                 (7)
                                                                                                                                                        (8)
    _get_signature_of = functools.partial(_signature_from_callable,=====================================================================================(9)
                                                                                                                                 so that it can use in itself
                                follow_wrapper_chains=follow_wrapper_chains,                                                                            (10)
                                skip_bound_arg=skip_bound_arg,                                                                                          (11)
print selected srcline with expands above----------
exec on 

In [18]:
#| column: screen

sig.dbprint(14, "obj must be callable")
sig.print(part=1)

                                sigcls=sigcls)                                                                                                          (12)
                                                                                                                                                        (13)
    if not callable(obj):===============================================================================================================================(14)
                                                                                                                                         obj must be callable
        raise TypeError('{!r} is not a callable object'.format(obj))                                                                                    (15)
                                                                                                                                                        (16)
print selected srcline with expands above----------
exec 

In [19]:
isinstance(Foo.clsmed, types.MethodType)

True

In [20]:
#| column: screen

sig.dbprint(17, "obj can be a classmethod")
sig.print(part=1)

        raise TypeError('{!r} is not a callable object'.format(obj))                                                                                    (15)
                                                                                                                                                        (16)
    if isinstance(obj, types.MethodType):===============================================================================================================(17)
                                                                                                                                     obj can be a classmethod
        # In this case we skip the first parameter of the underlying                                                                                    (18)
        # function (usually `self` or `cls`).                                                                                                           (19)
print selected srcline with expands above----------
exec 

In [21]:
from fastcore.meta import delegates

In [22]:
def low(a, b=1): pass
@delegates(low)
def mid(c, d=1, **kwargs): pass

In [23]:
#| column: screen

dbsig = sig.dbprint(28, "does Foo has __signature__?", "follow_wrapper_chains", \
            "obj = unwrap(obj, stop=(lambda f: hasattr(f, '__signature__')))", "isinstance(obj, types.MethodType)")

# inspect._signature_from_callable = _signature_from_callable
inspect._signature_from_callable = dbsig
# inspect._signature_from_callable = g['_signature_from_callable']

inspect.signature(mid)
sig.print(part=1) 

                                                                                                                                                        (26)
    # Was this function wrapped by a decorator?                                                                                                         (27)
    if follow_wrapper_chains:===========================================================================================================================(28)
                                                                                                                                  does Foo has __signature__?
        obj = unwrap(obj, stop=(lambda f: hasattr(f, "__signature__")))                                                                                 (29)
        if isinstance(obj, types.MethodType):                                                                                                           (30)
print selected srcline with expands above----------
exec 

In [24]:
#| column: screen

dbsig = sig.dbprint(37, "check __signature__", "obj = unwrap(obj, stop=(lambda f: hasattr(f, '__signature__')))", "obj.__signature__")
inspect._signature_from_callable = dbsig
inspect.signature(mid)
sig.print(part=2) 

                                                                                                                                                        (35)
    try:                                                                                                                                                (36)
        sig = obj.__signature__=========================================================================================================================(37)
                                                                                                                                          check __signature__
    except AttributeError:                                                                                                                              (38)
        pass                                                                                                                                            (39)
print selected srcline with expands above----------
exec 

### How exactly Foo get sig from `__init__`

In [25]:
#| column: screen

dbsig = sig.dbprint(91, "step 1: obj is a class?", "isinstance(obj, type)")
inspect._signature_from_callable = dbsig
inspect.signature(Foo)
sig.print(part=3) 



                                                                                                        follow_wrapper_chains => follow_wrapper_chains : True


                                   obj = unwrap(obj, stop=(lambda f: hasattr(f, '__signature__'))) => obj: <function IPythonKernel.do_execute at 0x1040c60d0>


                                                                               isinstance(obj, types.MethodType) => isinstance(obj, types.MethodType) : False


                                                                                                        follow_wrapper_chains => follow_wrapper_chains : True


                            obj = unwrap(obj, stop=(lambda f: hasattr(f, '__signature__'))) => obj: <function InteractiveShell.run_cell_async at 0x102956f70>


                                                                               isinstance(obj, types.MethodType) => isinstance(obj, types.MethodType) : False
                                        

In [26]:
#| column: screen

dbsig = sig.dbprint(96, "step 2: define its own __call__?", "call = _signature_get_user_defined_method(type(obj), '__call__')")
inspect._signature_from_callable = dbsig
inspect.signature(Foo)
sig.print(part=3) 



                                   obj = unwrap(obj, stop=(lambda f: hasattr(f, '__signature__'))) => obj: <function IPythonKernel.do_execute at 0x1040c60d0>




                            obj = unwrap(obj, stop=(lambda f: hasattr(f, '__signature__'))) => obj: <function InteractiveShell.run_cell_async at 0x102956f70>


        # First, let's see if it has an overloaded __call__ defined                                                                                     (94)
        # in its metaclass                                                                                                                              (95)
        call = _signature_get_user_defined_method(type(obj), '__call__')================================================================================(96)
                                                                                                                             step 2: define its own __call__?
        if call is not None:                   

In [27]:
#| column: screen

dbsig = sig.dbprint(101, "step 3: define its own __new__?", "new = _signature_get_user_defined_method(obj, '__new__')")
inspect._signature_from_callable = dbsig
inspect.signature(Foo)
sig.print(part=4)

        else:                                                                                                                                           (99)
            factory_method = None                                                                                                                       (100)
            new = _signature_get_user_defined_method(obj, '__new__')====================================================================================(101)
                                                                                                                              step 3: define its own __new__?
            init = _signature_get_user_defined_method(obj, '__init__')                                                                                  (102)
            # Now we check if the 'obj' class has an own '__new__' method                                                                               (103)
print selected srcline with expands above----------
e

In [28]:
#| column: screen

sig.dbprint(102, "step 4: define its own __init__?", "init = _signature_get_user_defined_method(obj, '__init__')")
inspect._signature_from_callable = dbsig
inspect.signature(Foo)
sig.print(part=4)

            factory_method = None                                                                                                                       (100)
            new = _signature_get_user_defined_method(obj, '__new__')                                                                                    (101)
            init = _signature_get_user_defined_method(obj, '__init__')==================================================================================(102)
                                                                                                                             step 4: define its own __init__?
            # Now we check if the 'obj' class has an own '__new__' method                                                                               (103)
            if '__new__' in obj.__dict__:                                                                                                               (104)
print selected srcline with expands above----------


In [29]:
#| column: screen

sig.dbprint(108, "step 5: __init__ is inside obj.__dict__?", "'__init__' in obj.__dict__")
inspect._signature_from_callable = dbsig
inspect.signature(Foo)
sig.print(part=4)

            # or an own '__init__' method                                                                                                               (106)
            elif '__init__' in obj.__dict__:                                                                                                            (107)
                factory_method = init===================================================================================================================(108)
                                                                                                                     step 5: __init__ is inside obj.__dict__?
            # If not, we take inherited '__new__' or '__init__', if present                                                                             (109)
            elif new is not None:                                                                                                                       (110)
print selected srcline with expands above----------


In [30]:
#| column: screen
dbsig = sig.dbprint(116, "step 6: run on itself using functools.partial", "sig = _get_signature_of(factory_method)")
inspect._signature_from_callable = dbsig
inspect.signature(Foo)

                                                                                                                                                        (114)
            if factory_method is not None:                                                                                                              (115)
                sig = _get_signature_of(factory_method)=================================================================================================(116)
                                                                                                                step 6: run on itself using functools.partial
                                                                                                                                                        (117)
        if sig is None:                                                                                                                                 (118)
print selected srcline with expands above----------


<Signature (a, b, c)>

In [31]:
#| column: screen
dbsig = sig.dbprint(76, "step 7: run on itself will run here", "isfunction(obj)", "_signature_is_functionlike(obj)")
inspect._signature_from_callable = dbsig
inspect.signature(Foo.__init__)

                return sig.replace(parameters=new_params)                                                                                               (74)
                                                                                                                                                        (75)
    if isfunction(obj) or _signature_is_functionlike(obj):==============================================================================================(76)
                                                                                                                          step 7: run on itself will run here
        # If it's a pure Python function, or an object that is duck type                                                                                (77)
        # of a Python function (Cython functions, for instance), then:                                                                                  (78)
print selected srcline with expands above----------
exec 

<Signature (self, a, b, c)>

In [32]:
#| column: screen
dbsig = sig.dbprint(79, "step 8: get sig with a different func", "env")
inspect._signature_from_callable = dbsig
inspect.signature(Foo)

        # If it's a pure Python function, or an object that is duck type                                                                                (77)
        # of a Python function (Cython functions, for instance), then:                                                                                  (78)
        return _signature_from_function(sigcls, obj,====================================================================================================(79)
                                                                                                                        step 8: get sig with a different func
                                        skip_bound_arg=skip_bound_arg)                                                                                  (80)
                                                                                                                                                        (81)
print selected srcline with expands above----------
exec 

<Signature (a, b, c)>

### Read commented `_signature_from_callable` from python 3.9+

In [33]:
#| column: screen
sig.print()



                                                                                                                    isfunction(obj) => isfunction(obj) : True


                                                                                    _signature_is_functionlike(obj) => _signature_is_functionlike(obj) : True


                                                                                                                    isfunction(obj) => isfunction(obj) : True


                                                                                    _signature_is_functionlike(obj) => _signature_is_functionlike(obj) : True
def _signature_from_callable(obj, *,======================================================(0)       
                             follow_wrapper_chains=True,==================================(1)       
                             skip_bound_arg=True,=========================================(2)       
                             sigcls):====================

## Foo's super class overriding `__new__` can stop Foo getting sig from `__init__`

Many things can go wrong to prevent a class to use the signature from `__init__`. 

FixSigMeta is a metaclass, which helps us to get our classes' signature right.

Then what types of the signature problems can FixSigMeta fix?

### When Foo's super class override `__new__`, python3.7 can't give Foo sig from `__init__`

1. when your class Foo inherits from class Base, if Base defines its `__new__`, then Foo can't get signature from `__init__`. (True for python 3.7 see [demos](https://www.kaggle.com/code/danielliao/notebook3edc928f49?scriptVersionId=104385507&cellId=1), no more for 3.9+)

In [34]:
class Base: # pass
    def __new__(self, **args): pass  # defines a __new__ 

class Foo(Base):
    def __init__(self, d, e, f): pass
    
inspect._signature_from_callable = sig.orisrc
inspect.signature(Foo) # no more problem for python 3.9+, 



                                                                                                                    isfunction(obj) => isfunction(obj) : True


                                                                                    _signature_is_functionlike(obj) => _signature_is_functionlike(obj) : True


                                                                                                                    isfunction(obj) => isfunction(obj) : True


                                                                                    _signature_is_functionlike(obj) => _signature_is_functionlike(obj) : True


<Signature (d, e, f)>

In [35]:
# from IPython.display import IFrame

# IFrame(src="https://www.kaggle.com/embed/danielliao/notebook3edc928f49?cellIds=2&kernelSessionId=104407182", width = "1200", height="300", \
#        style="margin: 0 auto; width: 100%; max-width: 950px;", frameborder="0", scrolling="auto", title="notebook3edc928f49")

In [36]:
inspect._signature_from_callable = _signature_from_callableOld
inspect.signature(Foo) # it is a problem for python 3.7, 

<Signature (**args)>

### How python3.7 and its inspect mess it up

In [37]:
# g = locals()

In [38]:
# sigOld = Fastdb(_signature_from_callableOld, g)
sigOld = Fastdb(_signature_from_callableOld)

In [39]:
#| column: screen

dbsigOld = sigOld.dbprint(115, "messup step 1: overriding __new__ is detected", "new = _signature_get_user_defined_method(obj, '__new__')")
inspect._signature_from_callable = dbsigOld 
print(inspect.signature(Foo)) # it is a problem for python 3.7, 
sigOld.print(part=4)

        else:                                                                                                                                           (113)
            # Now we check if the 'obj' class has a '__new__' method                                                                                    (114)
            new = _signature_get_user_defined_method(obj, '__new__')====================================================================================(115)
                                                                                                                messup step 1: overriding __new__ is detected
            if new is not None:                                                                                                                         (116)
                sig = _signature_from_callable(                                                                                                         (117)
print selected srcline with expands above----------


In [40]:
from pprint import pprint

In [41]:
#| column: screen

dbsigOld = sigOld.dbprint(117, "messup step 2: only __new__ sig is extracted", "env")
inspect._signature_from_callable = dbsigOld 
pprint(inspect.signature(Foo)) # it is a problem for python 3.7, 
sigOld.print(part=4)

            new = _signature_get_user_defined_method(obj, '__new__')                                                                                    (115)
            if new is not None:                                                                                                                         (116)
                sig = _signature_from_callable(=========================================================================================================(117)
                                                                                                                 messup step 2: only __new__ sig is extracted
                    new,                                                                                                                                (118)
                    follow_wrapper_chains=follow_wrapper_chains,                                                                                        (119)
print selected srcline with expands above----------


In [42]:
#| column: screen

dbsigOld = sigOld.dbprint(124, "messup step 3: __init__ don't even get accessed")
inspect._signature_from_callable = dbsigOld 
pprint(inspect.signature(Foo))
sigOld.print(part=4)

            else:                                                                                                                                       (122)
                # Finally, we should have at least __init__ implemented                                                                                 (123)
                init = _signature_get_user_defined_method(obj, '__init__')==============================================================================(124)
                                                                                                              messup step 3: __init__ don't even get accessed
                if init is not None:                                                                                                                    (125)
                    sig = _signature_from_callable(                                                                                                     (126)
print selected srcline with expands above----------


### FixSigMeta can fix it for python 3.7 inspect

Solution to 1: By also inheriting from the metaclass FixSigMeta can solve the signature problem for Foo (for python 3.7)

In [43]:
from fastcore.meta import FixSigMeta, test_sig

In [44]:
inspect._signature_from_callable = _signature_from_callableNew

In [45]:
class Base: # pass
    def __new__(self, **args): pass  # defines a __new__ 

class Foo(Base, metaclass=FixSigMeta):
    def __init__(self, d, e, f): pass
    
test_sig(Foo, '(d, e, f)')
inspect.signature(Foo)

<Signature (d, e, f)>

In [46]:
inspect._signature_from_callable = dbsigOld
inspect.signature(Foo) # No more a problem for python 3.7

<Signature (d, e, f)>

### How FixSigMeta fix it?

In [47]:
class Base: # pass
    def __new__(self, **args): pass  # defines a __new__ 

class Foo(Base):
    def __init__(self, d, e, f): pass
    
inspect._signature_from_callable = sigOld.orisrc
inspect.signature(Foo) 

<Signature (**args)>

In [48]:
hasattr(Foo, '__signature__')

False

In [49]:
class Base: # pass
    def __new__(self, **args): pass  # defines a __new__ 

class Foo(Base, metaclass=FixSigMeta):
    def __init__(self, d, e, f): pass
    
test_sig(Foo, '(d, e, f)')
inspect.signature(Foo)

<Signature (d, e, f)>

In [50]:
Foo.__signature__

<Signature (d, e, f)>

In [51]:
#| column: screen

dbsigOld = sigOld.dbprint(27, "FixSigMeta step 1: does it have __signature__", "env", "hasattr(obj, '__signature__')", \
               "obj = unwrap(obj, stop=(lambda f: hasattr(f, '__signature__')))", "inspect.getdoc(unwrap)")
inspect._signature_from_callable = dbsigOld
pprint(inspect.signature(Foo)) 
sigOld.print(part=1)

                                                                                                                                                        (25)
    # Was this function wrapped by a decorator?                                                                                                         (26)
    if follow_wrapper_chains:===========================================================================================================================(27)
                                                                                                                FixSigMeta step 1: does it have __signature__
        obj = unwrap(obj, stop=(lambda f: hasattr(f, "__signature__")))                                                                                 (28)
        if isinstance(obj, types.MethodType):                                                                                                           (29)
print selected srcline with expands above----------
exec 

In [52]:
#| column: screen

dbsigOld = sigOld.dbprint(44, "FixSigMeta step 2: use __signature__ as Foo's sig", "env", "sig = obj.__signature__", "isinstance(sig, Signature)")
inspect._signature_from_callable = dbsigOld
pprint(inspect.signature(Foo)) 
sigOld.print(part=2)

        pass                                                                                                                                            (42)
    else:                                                                                                                                               (43)
        if sig is not None:=============================================================================================================================(44)
                                                                                                            FixSigMeta step 2: use __signature__ as Foo's sig
            if not isinstance(sig, Signature):                                                                                                          (45)
                raise TypeError(                                                                                                                        (46)
print selected srcline with expands above----------
exec 

Note: new and old `_signature_from_callable` have the same code for getting signature for object with `__signature__`.

### Read commented `_signature_from_callable` of python 3.7

In [53]:
#| column: screen

sigOld.print()

def _signature_from_callableOld(obj, *,===================================================(0)       
                             follow_wrapper_chains=True,==================================(1)       
                             skip_bound_arg=True,=========================================(2)       
                             sigcls):=====================================================(3)       
                                                                                                                                                        (4)
    """Private helper function to get signature for arbitrary=============================(5)       
    callable objects.=====================================================================(6)       
    """===================================================================================(7)       
                                                                                                                                         

## Foo's metaclass defines its own `__call__` will stop Foo get sig from `__init__`

### Problem demo

In [54]:
inspect._signature_from_callable = _signature_from_callableNew

2. when your Foo has a metaclass BaseMeta, if BaseMeta need to define its `__call__`, then Foo can't get signature from `__init__`.

In [55]:
class BaseMeta(type): 
    # using __new__ from type
    def __call__(cls, *args, **kwargs): pass
class Foo(metaclass=BaseMeta): 
    def __init__(self, d, e, f): pass

test_sig(Foo, '(*args, **kwargs)')

In [56]:
class BaseMeta(type): 
    def __new__(cls, name, bases, dict):
        return super().__new__(cls, name, bases, dict) # using __new__ from type
    def __call__(cls, *args, **kwargs): pass
class Foo(metaclass=BaseMeta): 
    def __init__(self, d, e, f): pass

test_sig(Foo, '(*args, **kwargs)')

### Cause of the problem

Now I have a better understanding of the source codes, I have 2 places to investigate, they are roughly at line  96 and line 37.

In [57]:
#| column: screen

dbsig = sig.dbprint(37, "does Foo store sig inside __signature__", "hasattr(obj, '__signature__')")
inspect._signature_from_callable = dbsig
pprint(inspect.signature(Foo))
sig.print(part=2)

                                                                                                                                                        (35)
    try:                                                                                                                                                (36)
        sig = obj.__signature__=========================================================================================================================(37)
                                                                                                                      does Foo store sig inside __signature__
    except AttributeError:                                                                                                                              (38)
        pass                                                                                                                                            (39)
print selected srcline with expands above----------
exec 

In [58]:
#| column: screen

dbsig = sig.dbprint(98, "__call__ is defined", "sig = _get_signature_of(call)")
inspect._signature_from_callable = dbsig
pprint(inspect.signature(Foo))
sig.print(part=3)

        call = _signature_get_user_defined_method(type(obj), '__call__')                                                                                (96)
        if call is not None:                                                                                                                            (97)
            sig = _get_signature_of(call)===============================================================================================================(98)
                                                                                                                                          __call__ is defined
        else:                                                                                                                                           (99)
            factory_method = None                                                                                                                       (100)
print selected srcline with expands above----------
exec

### Solution demo

Solution to problem 2: you need to inherit from FixSigMeta instead of type when constructing the metaclass to preserve the signature in `__init__`. Be careful not to override `__new__` when doing this:

In [59]:
class BaseMeta(FixSigMeta): 
    # using __new__ of  FixSigMeta instead of type
    def __call__(cls, *args, **kwargs): pass

class Foo(metaclass=BaseMeta): # Base
    def __init__(self, d, e, f): pass

test_sig(Foo, '(d, e, f)')



                                                                                       hasattr(obj, '__signature__') => hasattr(obj, '__signature__') : False


                                                                                       hasattr(obj, '__signature__') => hasattr(obj, '__signature__') : False


In [60]:
class BaseMeta(FixSigMeta): 
    def __new__(cls, name, bases, dict): # not really overriding __new__, still using FixSigMeta.__new__ actually
        return super().__new__(cls, name, bases, dict)
    def __call__(cls, *args, **kwargs): pass

class Foo(metaclass=BaseMeta): # Base
    def __init__(self, d, e, f): pass

test_sig(Foo, '(d, e, f)')



                                                                                       hasattr(obj, '__signature__') => hasattr(obj, '__signature__') : False


                                                                                       hasattr(obj, '__signature__') => hasattr(obj, '__signature__') : False


Note: if Base also defines `__new__`, then FixSigMeta can't help. 

### How FixSigMeta fix this problem

In [61]:
#| column: screen

dbsig = sig.dbprint(29, "why has to unwrap?", "hasattr(obj, '__signature__')")
inspect._signature_from_callable = dbsig
pprint(inspect.signature(Foo))
sig.print(part=1)



                                                                                       hasattr(obj, '__signature__') => hasattr(obj, '__signature__') : False


                                                                                       hasattr(obj, '__signature__') => hasattr(obj, '__signature__') : False
    # Was this function wrapped by a decorator?                                                                                                         (27)
    if follow_wrapper_chains:                                                                                                                           (28)
        obj = unwrap(obj, stop=(lambda f: hasattr(f, "__signature__")))=================================================================================(29)
                                                                                                                                           why has to unwrap?
        if isinstance(obj, types.MethodType):      

In [62]:
#| column: screen

dbsig = sig.dbprint(30, "what is wrapped by Foo?", "isinstance(obj, types.MethodType)", "obj", "type(obj)")
inspect._signature_from_callable = dbsig
pprint(inspect.signature(Foo))
sig.print(part=2)

    if follow_wrapper_chains:                                                                                                                           (28)
        obj = unwrap(obj, stop=(lambda f: hasattr(f, "__signature__")))                                                                                 (29)
        if isinstance(obj, types.MethodType):===========================================================================================================(30)
                                                                                                                                      what is wrapped by Foo?
            # If the unwrapped object is a *method*, we might want to                                                                                   (31)
            # skip its first parameter (self).                                                                                                          (32)
print selected srcline with expands above----------
exec 

### Common feature of the solutions above by FixSigMeta

The key is to create `__signature__` for Foo, so that `inspect.signature` will get sig from `__signature__`, instead of `__new__` or `__call__`

In [67]:
from fastcore.meta import FixSigMeta



                                                                                       hasattr(obj, '__signature__') => hasattr(obj, '__signature__') : False


                                                                                       hasattr(obj, '__signature__') => hasattr(obj, '__signature__') : False


In [75]:
import fastcore.meta as fm



                                                                                       hasattr(obj, '__signature__') => hasattr(obj, '__signature__') : False


                                                                                       hasattr(obj, '__signature__') => hasattr(obj, '__signature__') : False


In [76]:
fm.__dict__



                                                                                       hasattr(obj, '__signature__') => hasattr(obj, '__signature__') : False


                                                                                       hasattr(obj, '__signature__') => hasattr(obj, '__signature__') : False


                                                                               isinstance(obj, types.MethodType) => isinstance(obj, types.MethodType) : False


                                                                                                             obj => obj : <built-in function __build_class__>


                                                                                                type(obj) => type(obj) : <class 'builtin_function_or_method'>


                                                                               isinstance(obj, types.MethodType) => isinstance(obj, types.MethodType) : False


                                      

{'__name__': 'fastcore.meta',
 '__doc__': None,
 '__package__': 'fastcore',
 '__loader__': <_frozen_importlib_external.SourceFileLoader at 0x10461be50>,
 '__spec__': ModuleSpec(name='fastcore.meta', loader=<_frozen_importlib_external.SourceFileLoader object at 0x10461be50>, origin='/Users/Natsume/mambaforge/lib/python3.9/site-packages/fastcore/meta.py'),
 '__file__': '/Users/Natsume/mambaforge/lib/python3.9/site-packages/fastcore/meta.py',
 '__cached__': '/Users/Natsume/mambaforge/lib/python3.9/site-packages/fastcore/__pycache__/meta.cpython-39.pyc',
 '__builtins__': {'__name__': 'builtins',
  '__doc__': "Built-in functions, exceptions, and other objects.\n\nNoteworthy: None is the `nil' object; Ellipsis represents `...' in slices.",
  '__package__': '',
  '__loader__': _frozen_importlib.BuiltinImporter,
  '__spec__': ModuleSpec(name='builtins', loader=<class '_frozen_importlib.BuiltinImporter'>, origin='built-in'),
  '__build_class__': <function __build_class__>,
  '__import__': <func

In [72]:
eval('fastcore.meta')



                                                                                       hasattr(obj, '__signature__') => hasattr(obj, '__signature__') : False


                                                                                       hasattr(obj, '__signature__') => hasattr(obj, '__signature__') : False


<module 'fastcore.meta' from '/Users/Natsume/mambaforge/lib/python3.9/site-packages/fastcore/meta.py'>

In [74]:
delegates.__dict__



                                                                                       hasattr(obj, '__signature__') => hasattr(obj, '__signature__') : False


                                                                                       hasattr(obj, '__signature__') => hasattr(obj, '__signature__') : False


{}

In [63]:
# g = locals()
# fsm = Fastdb(FixSigMeta, g)
fsm = Fastdb(FixSigMeta)



                                                                                       hasattr(obj, '__signature__') => hasattr(obj, '__signature__') : False


                                                                                       hasattr(obj, '__signature__') => hasattr(obj, '__signature__') : False


AttributeError: type object 'FixSigMeta' has no attribute '__globals__'

In [ ]:
from fastcore.meta import _rm_self

In [ ]:
#| column: screen

dbfsm = fsm.dbprint(4, "FixSigMeta create Foo with its __new__", "res", "inspect.signature(res.__init__)", "_rm_self(inspect.signature(res.__init__))")
FixSigMeta = dbfsm
inspect._signature_from_callable = sig.orisrc # deactivate it

class BaseMeta(FixSigMeta): 
    def __new__(cls, name, bases, dict): # not really overriding __new__, still using FixSigMeta.__new__ actually
        return super().__new__(cls, name, bases, dict)
    def __call__(cls, *args, **kwargs): pass

class Foo(metaclass=BaseMeta): # Base
    def __init__(self, d, e, f): pass

test_sig(Foo, '(d, e, f)')

In [ ]:
fsm.print()

In [ ]:
FixSigMeta = fsm.orisrc

In [ ]:
from fastcore.meta import test_eq

In [ ]:
class BaseMeta(FixSigMeta): 
    # __new__ comes from FixSigMeta
    def __new__(cls, *args, **kwargs): pass # as it create None for Foo, there is no signature neither
    def __call__(cls, *args, **kwargs): pass

class Foo(metaclass=BaseMeta): # Base
    def __init__(self, d, e, f): pass

test_eq(type(Foo), type(None))

Note: if Base also defines `__init__`, then FixSigMeta can still help. 

In [ ]:
class BaseMeta(FixSigMeta): 
    # __new__ comes from FixSigMeta
    def __init__(cls, *args, **kwargs): pass # this __init__ is not used by Foo
    def __call__(cls, *args, **kwargs): pass

class Foo(metaclass=BaseMeta): # Base
    def __init__(self, d, e, f): pass # override the __init__ above

test_sig(Foo, '(d, e, f)')